<a href="https://colab.research.google.com/github/blusewill/ytvideo-whisper/blob/master/ytvideo_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to the Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
! mkdir /content/drive/MyDrive/Subtitle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/MyDrive/Subtitle’: File exists


## Install the Whisper Code and yt-dlp

In [ ]:
! pip install openai-whisper
! pip install yt-dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 17.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796926 sha256=2ab4ffc7152273f285c0ad7fab8725a85759394a512525bff24e1edcb2039fef
  Stored in directory: /root/.cache/pip/wheels/c4/85/e6/0bb9507b8e4f3f6d9c6dcf318bc3514739430375aa8e9eaf5b
Successfully built openai-whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Check we have a GPU

You should see the output `device(type='cuda', index=0)` below. If you don't, you may be on a CPU-only Colab instance which will run more slowly. Go to `Runtime->Change Runtime Type` to fix this.

In [ ]:
import whisper

model = whisper.load_model("medium")

print("Whisper model loaded.")

model.device

100%|█████████████████████████████████████| 1.42G/1.42G [00:26<00:00, 57.6MiB/s]


Whisper model loaded.


device(type='cuda', index=0)

## Download Audio Files

This repository has a couple of pre-recorded MP3s to run through the transcribe function. You can listen to them with the audio widgets displayed below.

In [ ]:
import yt_dlp

links = input("Please type your YouTube Video Link Here and Press Enter : ")
name = input("Please type your SRT file name Here and Press Enter : ")

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
        'outtmpl': 'content/audio',
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([links])


Please type your YouTube Video Link Here and Press Enter : https://www.youtube.com/watch?v=tD2fBWsZrZU
Please type your SRT file name Here and Press Enter : 日麻小教室2
[youtube] Extracting URL: https://www.youtube.com/watch?v=tD2fBWsZrZU
[youtube] tD2fBWsZrZU: Downloading webpage
[youtube] tD2fBWsZrZU: Downloading android player API JSON
[youtube] tD2fBWsZrZU: Downloading player 248ded94
[info] tD2fBWsZrZU: Downloading 1 format(s): 251
[dashsegments] Total fragments: 11
[download] Destination: content/audio
[download] 100% of  108.01MiB in 00:00:27 at 3.88MiB/s                 
[ExtractAudio] Destination: content/audio.mp3
Deleting original file content/audio (pass -k to keep)


# Start Generating Srt File

In [ ]:
from whisper.utils import get_writer

filename = "audio.mp3"
input_directory = "content"
input_file = f"{input_directory}/{filename}"

result = model.transcribe(input_file)
transcription_root = "/content/drive/MyDrive/Subtitle"

# Save as an SRT file
srt_writer = get_writer("srt", transcription_root,)
srt_writer(result, name,)